# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл.

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Документация: https://requests.readthedocs.io/en/latest/

Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [35]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [36]:
page  # response 200 - страница загружена

<Response [200]>

**BeautifulSoup** - python-библиотека для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы.

Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [ ]:
!pip install bs4

In [37]:
from bs4 import BeautifulSoup 
# название - отсылка к песне про суп из Алисы в стране чудес https://aliceinwonderland.fandom.com/wiki/Turtle_Soup

In [38]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [39]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1 — главное издание о науке, технике и технологиях</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=c066269c9e31b8c5719ba747306ba601" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=ab07a66910bee48e90bcb5a042d79cad" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#f26e40" href=

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [40]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1 — главное издание о науке, технике и технологиях
  </title>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
  <link href="/front-build/css/main.css?id=c066269c9e31b8c5719ba747306ba601" rel="stylesheet"/>
  <link href="/front-build/css/app.css?id=ab07a66910bee48e90bcb5a042d79cad" rel="stylesheet"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/site.webmanifest" rel="manifest"

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. 

**Функция `soup.find('a')`** найдет первый в дереве тег \<a>.
Если нам нужно найти не только первый элемент, а все элементы по определенному признаку, следует использовать функцию **`soup.find_all('a')`**

In [43]:
soup.find_all('a')

[<a class="w-7.5 h-7.5" href="/search">
 <svg class="w-full h-full">
 <use xlink:href="#n1_pupa"></use>
 </svg>
 </a>,
 <a class="col-span-3 lg:col-span-2 col-start-7 lg:col-start-9 flex items-center hover:text-main transition-colors duration-75" href="https://offline.nplus1.ru/">N + offline</a>,
 <a class="col-span-3 lg:col-span-2 col-start-10 lg:col-start-11 flex items-center hover:text-main transition-colors duration-75" href="https://nplus.pro/">N + production</a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/about">
                 О нас
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/difficult" target="_blank">
                 Сложность
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/adv" target="_blank">
                 Рекламодателям
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus

In [42]:
len(soup.find_all('a'))

436

In [44]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https://nplus1.ru/search?tags=874
https://nplus1.ru/search?tags=880
https://nplus1.ru/search?tags=768
https://nplus1.ru/search?tags=890
https://nplus1.ru/search?tags=871
https://nplus1.ru/search?tags=876
https://nplus1.ru/search?tags=775
https://nplus1.ru/search?tags=767
https://nplus1.ru/search?tags=771
https://nplus1.ru/search?tags=772
https://nplus1.ru/search?tags=778
https://nplus1.ru/search?tags=917
https://nplus1.ru/search?tags=918
https://nplus1.ru/search?tags=824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/news/2015/09/21/editor-thy-name
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=8

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [45]:
urls = []

for link in soup.find_all('a'):
#     print(link)
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/10/04/chem-nobel-2023',
 'https://nplus1.ru/news/2023/10/05/manganese-accumulation-in-the-brain',
 'https://nplus1.ru/news/2023/10/04/sulfur-of-deccan-traps',
 'https://nplus1.ru/news/2023/10/04/heartbreaking',
 'https://nplus1.ru/news/2023/10/04/osiris-rex-deploy-capsule',
 'https://nplus1.ru/news/2023/10/04/vehicle-crash-adhd',
 'https://nplus1.ru/news/2023/10/04/same-sex-behaviour-mammals',
 'https://nplus1.ru/news/2023/10/04/frb-gw-or-no',
 'https://nplus1.ru/news/2023/10/04/ak-alakha',
 'https://nplus1.ru/news/2023/10/04/second-vaccine-for-malaria',
 'https://nplus1.ru/news/2023/10/04/chem-nobel-2023',
 'https://nplus1.ru/news/2023/10/04/neolithic-jericho',
 'https://nplus1.ru/news/2023/10/04/we-drive-solar-sharing-batteries',
 'https://nplus1.ru/news/2023/10/04/orca-vs-sea-otters',
 'https://nplus1.ru/news/2023/10/03/wolves-love-story',
 'https://nplus1.ru/news/2023/10/03/bivalve-cancer',
 'https:

In [46]:
len(urls)

55

In [49]:
urls[1]

'https://nplus1.ru/news/2023/10/04/chem-nobel-2023'

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [50]:
url0 = urls[1]

page0 = requests.get(url0)
print(page0)

soup0 = BeautifulSoup(page0.text, 'html')
print(soup0)

<Response [200]>
<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Нобелевскую премию по химии присудили за квантовые точки. Ее получат Мунги Бавенди, Луис Брюс и Алексей Екимов</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=c066269c9e31b8c5719ba747306ba601" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=ab07a66910bee48e90bcb5a042d79cad" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<l

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [53]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Нобелевскую премию по химии присудили за квантовые точки" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/838971/6516cd4f91d3b_cover_share.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2023/10/04/chem-nobel-2023" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content=

In [55]:
soup0.find('meta', {'property': 'og:title'}).get('content')

'Нобелевскую премию по\xa0химии присудили за\xa0квантовые точки'

In [56]:
soup0.find_all('meta', {'itemprop': 'datePublished'})[0].get('content')

'2023-10-04'

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [61]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Михаил Бойм" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [17]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Михаил Бойм" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [64]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Михаил Бойм'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [66]:
author = soup0.find('meta', {'name' : 'author'}).get('content')
author

'Михаил Бойм'

Аналогичным образом извлечем дату, заголовок и описание.

In [67]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

In [68]:
title

'Нобелевскую премию по\xa0химии присудили за\xa0квантовые точки'

In [69]:
date

'2023-10-04'

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [70]:
def GetNews(url0):
    """
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'html')
    
    author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
     
    return url0, date, author, title

In [71]:
help(GetNews)

Help on function GetNews in module __main__:

GetNews(url0)
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).



Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [72]:
from time import sleep

In [26]:
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/10/04/chem-nobel-2023',
 'https://nplus1.ru/news/2023/10/04/vehicle-crash-adhd',
 'https://nplus1.ru/news/2023/10/04/same-sex-behaviour-mammals',
 'https://nplus1.ru/news/2023/10/04/frb-gw-or-no',
 'https://nplus1.ru/news/2023/10/04/ak-alakha',
 'https://nplus1.ru/news/2023/10/04/second-vaccine-for-malaria',
 'https://nplus1.ru/news/2023/10/04/chem-nobel-2023',
 'https://nplus1.ru/news/2023/10/04/neolithic-jericho',
 'https://nplus1.ru/news/2023/10/04/we-drive-solar-sharing-batteries',
 'https://nplus1.ru/news/2023/10/04/orca-vs-sea-otters',
 'https://nplus1.ru/news/2023/10/03/wolves-love-story',
 'https://nplus1.ru/news/2023/10/03/bivalve-cancer',
 'https://nplus1.ru/news/2023/10/03/butterflies-plugs',
 'https://nplus1.ru/news/2023/10/03/gwas-carrot',
 'https://nplus1.ru/news/2023/10/03/phys-nobel-2023',
 'https://nplus1.ru/news/2023/10/03/yamnaya-people-and-multiple-sclerosis',
 'https://nplus1.ru/new

In [73]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[1:]:
    print(link)
    res = GetNews(link)
    news.append(res)
        
    sleep(3) # задержка в 3 секунды

https://nplus1.ru/news/2023/10/04/chem-nobel-2023
https://nplus1.ru/news/2023/10/05/manganese-accumulation-in-the-brain
https://nplus1.ru/news/2023/10/04/sulfur-of-deccan-traps
https://nplus1.ru/news/2023/10/04/heartbreaking
https://nplus1.ru/news/2023/10/04/osiris-rex-deploy-capsule
https://nplus1.ru/news/2023/10/04/vehicle-crash-adhd
https://nplus1.ru/news/2023/10/04/same-sex-behaviour-mammals
https://nplus1.ru/news/2023/10/04/frb-gw-or-no
https://nplus1.ru/news/2023/10/04/ak-alakha
https://nplus1.ru/news/2023/10/04/second-vaccine-for-malaria
https://nplus1.ru/news/2023/10/04/chem-nobel-2023
https://nplus1.ru/news/2023/10/04/neolithic-jericho
https://nplus1.ru/news/2023/10/04/we-drive-solar-sharing-batteries
https://nplus1.ru/news/2023/10/04/orca-vs-sea-otters
https://nplus1.ru/news/2023/10/03/wolves-love-story
https://nplus1.ru/news/2023/10/03/bivalve-cancer
https://nplus1.ru/news/2023/10/03/butterflies-plugs
https://nplus1.ru/news/2023/10/03/gwas-carrot
https://nplus1.ru/news/2023/

In [74]:
news

[('https://nplus1.ru/news/2023/10/04/chem-nobel-2023',
  '2023-10-04',
  'Михаил Бойм',
  'Нобелевскую премию по\xa0химии присудили за\xa0квантовые точки'),
 ('https://nplus1.ru/news/2023/10/05/manganese-accumulation-in-the-brain',
  '2023-10-05',
  'Слава Гоменюк',
  'Врачи описали хроническое отравление марганцем из-за сварочных работ'),
 ('https://nplus1.ru/news/2023/10/04/sulfur-of-deccan-traps',
  '2023-10-04',
  'Винера Андреева',
  'Извержения Деканских траппов принесли динозаврам серию вулканических зим перед ударом астероида'),
 ('https://nplus1.ru/news/2023/10/04/heartbreaking',
  '2023-10-04',
  'Олег Лищук',
  'Злоупотребление каннабисом связали с\xa0повышенным риском болезней сердца и\xa0сосудов'),
 ('https://nplus1.ru/news/2023/10/04/osiris-rex-deploy-capsule',
  '2023-10-04',
  'Александр Войтюк',
  'NASA показало полет возвращаемой капсулы станции OSIRIS-REx к\xa0Земле'),
 ('https://nplus1.ru/news/2023/10/04/vehicle-crash-adhd',
  '2023-10-04',
  'Слава Гоменюк',
  'Пож

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [75]:
import pandas as pd

In [76]:
df = pd.DataFrame(news)

In [77]:
df.head(10)

,0,1,2,3
0,https://nplus1.ru/news/2023/10/04/chem-nobel-2023,2023-10-04,Михаил Бойм,Нобелевскую премию по химии присудили за квант...
1,https://nplus1.ru/news/2023/10/05/manganese-ac...,2023-10-05,Слава Гоменюк,Врачи описали хроническое отравление марганцем...
2,https://nplus1.ru/news/2023/10/04/sulfur-of-de...,2023-10-04,Винера Андреева,Извержения Деканских траппов принесли динозавр...
3,https://nplus1.ru/news/2023/10/04/heartbreaking,2023-10-04,Олег Лищук,Злоупотребление каннабисом связали с повышенны...
4,https://nplus1.ru/news/2023/10/04/osiris-rex-d...,2023-10-04,Александр Войтюк,NASA показало полет возвращаемой капсулы станц...
5,https://nplus1.ru/news/2023/10/04/vehicle-cras...,2023-10-04,Слава Гоменюк,Пожилые водители с СДВГ отличились повышенным ...
6,https://nplus1.ru/news/2023/10/04/same-sex-beh...,2023-10-04,Катерина Петрова,Однополое сексуальное поведение млекопитающих ...
7,https://nplus1.ru/news/2023/10/04/frb-gw-or-no,2023-10-04,Александр Войтюк,Астрономы не нашли связи между быстрыми радиов...
8,https://nplus1.ru/news/2023/10/04/ak-alakha,2023-10-04,Михаил Подрезов,Знатному пазырыкцу поставили диффузный идиопат...
9,https://nplus1.ru/news/2023/10/04/second-vacci...,2023-10-04,Слава Гоменюк,ВОЗ одобрила вторую противомалярийную вакцину ...


Переименуем столбцы в базе.

In [78]:
df.columns = ['link', 'date', 'author', 'title']

In [79]:
df.head()

,link,date,author,title
0,https://nplus1.ru/news/2023/10/04/chem-nobel-2023,2023-10-04,Михаил Бойм,Нобелевскую премию по химии присудили за квант...
1,https://nplus1.ru/news/2023/10/05/manganese-ac...,2023-10-05,Слава Гоменюк,Врачи описали хроническое отравление марганцем...
2,https://nplus1.ru/news/2023/10/04/sulfur-of-de...,2023-10-04,Винера Андреева,Извержения Деканских траппов принесли динозавр...
3,https://nplus1.ru/news/2023/10/04/heartbreaking,2023-10-04,Олег Лищук,Злоупотребление каннабисом связали с повышенны...
4,https://nplus1.ru/news/2023/10/04/osiris-rex-d...,2023-10-04,Александр Войтюк,NASA показало полет возвращаемой капсулы станц...


Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [81]:
df.to_excel('nplus-news.xlsx', index=False)